In [ ]:
import numpy as np

import cleaning as cl
import tuning as tn

seed = 1337

## Read in data

In [ ]:
import pandas as pd

train_df = pd.read_csv('train.csv')
X_eval = pd.read_csv('test.csv')

X = train_df.drop(columns='SalePrice')
y = train_df.SalePrice

## Clean up data for model fitting

In [ ]:
from sklearn.model_selection import train_test_split

X, X_eval = cl.create_one_hot_encoding(X, y, X_eval)
# X_t, X_v, y_t, y_v = train_test_split(X, y, 
#                                       test_size=.2,
#                                       random_state=seed)

## Create models

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

models = {'rfr': RandomForestRegressor(n_estimators=100, 
                                       criterion='mae', 
                                       n_jobs=-1,
                                       random_state=seed),
          'xgbr': XGBRegressor(n_estimators=100,
                               random_state=seed), 
          'skgbr': GradientBoostingRegressor(loss='lad', 
                                             n_estimators=100,
                                             random_state=seed)}

## Pipeline models into estimators

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

estimators = {name: make_pipeline(SimpleImputer(), model) 
              for name, model in models.items()}

## Fit estimators to training data

In [ ]:
fit_params = {'rfr': {},
              'xgbr': {'xgbregressor__verbose': False}, 
              'skgbr': {}}

In [ ]:
for name, params in fit_params.items():
    estimators[name].fit(X_t, y_t, **params)

In [ ]:
from sklearn.metrics import mean_absolute_error

for name, estimator in estimators.items():
    y_pred = estimator.predict(X_v)

    print(mean_absolute_error(y_pred, y_v))

## Get cross validation scores

In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = {}

for name, estimator in estimators.items():
    cv_scores[name] = cross_val_score(estimator, X, y, cv=5,
                                      scoring='neg_mean_absolute_error', fit_params=fit_params[name])
    
    print(f'{name} cv score: {-np.mean(cv_scores[name])}')

## This works for cv yeet

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from scipy.stats import uniform
from scipy.stats import reciprocal

In [ ]:
xgbr_params = {'xgbregressor__n_estimators': randint(low=1e2, high=1e3),
               'xgbregressor__learning_rate': reciprocal(a=1e-2, b=1e-1)}

In [ ]:
results = RandomizedSearchCV(estimators['xgbr'], param_distributions=xgbr_params, 
                   n_iter=10, scoring='neg_mean_absolute_error', cv=5).fit(X,y)

In [ ]:
results.best_params_

In [ ]:
randint(1, 10).interval(1)

## Steps for auto param search

1. Use `RandomizedSearchCV` to find best params over a range.

2. Take results of `RandomizedSearchCV` to reduce area of grid to search.
    + Search should happen around the best param, and the area should be reduced by a factor.
    + Add parameter specifying number of iterations of `RandomizedSearchCV` to run, each with the reduced parameter grid area.

3. Return the parameters, the score, and the model that is the best fit.

In [ ]:
def make_param_set(low, high, n, is_log=False, is_int=False):
    if is_log:
        low, high = [np.log(x) for x in [low, high]]
    
    step_size = (high - low) / (n-1)
    param_set = [low + step_size * i for i in range(n)]
    
    if is_log:
        param_set = [np.exp(p) for p in param_set]
        
    if is_int:
        param_set = [int(p) for p in param_set]
        
    return param_set

In [ ]:
def make_all_param_sets(param_ranges):
    param_sets = {}
    for args in param_ranges:
        param_set = make_param_set(args['low'], args['high'], args['n'], args['is_log'], args['is_int'])
        param_sets[args['name']] = param_set
    
    return param_sets

In [ ]:
def find_hyperparams_iterated(model, X, y, param_ranges, n_samplings, scoring, cv_folds, n_iters, alpha):
    """
    Find best hyperparams based on cross-validation scores.
    
    param_ranges: list of param arguments to build a param set.
    """
    param_sets = make_all_param_sets(param_ranges)
    
    for i in range(n_iters):
        rand_search_results = RandomizedSearchCV(model, param_distributions=param_distributions,
                                                 n_iter=n_samplings, scoring=scoring, cv=cv_folds).fit(X,y)
        

In [ ]:
make_all_param_sets([{'name': 'xgbregressor__n_estimators', 
                      'low': 1e2, 
                      'high': 1e3, 
                      'n': 10, 
                      'is_log': False, 
                      'is_int': True},
                     {'name': 'xgbregressor__learning_rate',
                      'low': 1e-2,
                      'high': 1e-1,
                      'n': 10,
                      'is_log': True,
                      'is_int': False}])

In [ ]:
make_param_set(.01, .1, 10, True, False)

In [ ]:
def reduce_param_range(low, high, param, alpha, is_int=True):
    param_range = high - low
    low, high = (param - alpha * param_range / 2, param + alpha * param_range / 2)
    
    if is_int:
        low, high = [int(x) for x in [low, high]]
        
    return low, high

In [ ]:
reduce_param_range(1, 1000, 900, .5)

## Use cv for hyperparameter tuning: XGBR

Let's try to choose the optimal value for:

1. `n_estimators`
2. `learning_rate`

TODO: Automate hyperparameter search

In [ ]:
n_est_set = np.random.randint(600, 1000, 10)
lr_set = np.random.uniform(.03, .06, 10)

cv_scores = []

for n_est, lr in zip(n_est_set, lr_set):
    xgbr = make_pipeline(SimpleImputer(), 
                         XGBRegressor(n_estimators=n_est, learning_rate=lr, random_state=seed),
                         memory=cachedir)
    cv_score = cross_val_score(xgbr, X, y, cv=5,
                               scoring='neg_mean_absolute_error')
    cv_scores.append({'cv_score': -np.mean(cv_score), 'n_est': n_est, 'lr': lr})
    print(cv_scores[-1])
    
min(cv_scores, key=lambda x: x['cv_score'])

In [ ]:
from tempfile import mkdtemp
from shutil import rmtree

def pick_n_est_lr(n_est_range, lr_range, n):
    n_est_set = np.random.randint(*n_est_range, n)
    lr_set = np.random.uniform(*lr_range, n)
    
    cv_score = []
    
    cachedir = mkdtemp()
    for n_est, lr in zip(n_est_set, lr_set):
        xgbr = make_pipeline(SimpleImputer(), 
                             XGBRegressor(n_estimators=n_est, learning_rate=lr, random_state=seed),
                             memory=cachedir)
        cv_score = cross_val_score(xgbr, X, y, cv=5,
                                   scoring='neg_mean_absolute_error')
        cv_scores.append({'cv_score': -np.mean(cv_score), 'n_est': n_est, 'lr': lr})
    rmtree(cachedir)
    
    return min(cv_scores, key=lambda x: x['cv_score'])

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
?GridSearchCV

In [ ]:
from importlib import reload
reload(tn)

In [ ]:
hyperparams = tn.pick_xgbr_hyperparams(X, y, (100, 1000), (.1, .2), 25)

### Make predictions on test set

In [ ]:
y_pred_test = (make_pipeline(SimpleImputer(),
                             XGBRegressor(n_estimators=842, learning_rate=.05377, random_state=seed))
               .fit(X,y)
               .predict(X_eval))

In [ ]:
out = pd.DataFrame({'Id': X_eval.Id.astype(int), 'SalePrice': y_pred_test})
out.to_csv('xgbr_submission.csv', index=False)

### Make partial dependence plots

In [ ]:
from sklearn.ensemble.partial_dependence import plot_partial_dependence
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
bgbt = GradientBoostingRegressor(n_estimators=300, loss='lad')
bgbt.fit(X_t, y_t)

In [ ]:
important_features = ['LotArea', 'BedroomAbvGr', 'OverallCond', 'TotRmsAbvGrd']
features_indices = [X_t.columns.get_loc(f) for f in important_features]

In [ ]:
plot_partial_dependence(bgbt, X_t, [0,1], important_features)

In [ ]:
plot_partial_dependence(bgbt, X_t, [2,3], important_features)